In [1]:
import os

project_folder = r"C:\Users\ozgur\Desktop\Final Project\Bitirme Projesi\Fotoğraflar"
spectrogram_folder = os.path.join(project_folder, '5_height_and_congruence_of_the_medial_longitudinal_arch')

In [5]:
import cv2

IMG_HEIGHT = 64
IMG_WIDTH = 64
dataset = []

def get_images_labels(file_folder):
  for root, directory, filenames in os.walk(file_folder):
    for filename in filenames:
      if filename.endswith("jpg"):
        read_images = cv2.imread(os.path.join(root, filename), 1)  # 0: cv2.IMREAD_GRAYSCALE; 1: cv2.IMREAD_COLOR; -1: cv2.IMREAD_UNCHANGED    
        if(type(read_images) == type(None)):
          pass
        else:
          read_images = cv2.resize(read_images, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_AREA)                                      
        # read_images = cv2.resize(read_images, dsize=(IMG_WIDTH, IMG_HEIGHT),interpolation=cv2.INTER_AREA)  # resizing; interpolation=cv2.INTER_AREA
        read_images = cv2.cvtColor(read_images, cv2.COLOR_BGR2GRAY)  #grayscaling
        label_name = os.path.basename(root)
        grayscale_images = read_images / 255  # normalizing
        dataset.append([grayscale_images, label_name])
  return dataset

In [15]:
def get_images_labels(file_folder):
  for root, directory, filenames in os.walk(file_folder):
    for filename in filenames:
      if filename.endswith("jpg"):
        read_images = cv2.imread(os.path.join(root, filename), 1)  # 0: cv2.IMREAD_GRAYSCALE; 1: cv2.IMREAD_COLOR; -1: cv2.IMREAD_UNCHANGED    
        if(type(read_images) == type(None)):
          pass
        else:
          read_images = cv2.resize(read_images, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_AREA)   
          
    return dataset

In [18]:
get_images_labels(spectrogram_folder)

[]

In [16]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

def data_preprocessing():
  encoder = LabelEncoder()
  obj_data = get_images_labels(spectrogram_folder)

  image_data = np.array([obj_data[image_loc][0] for image_loc in range(len(obj_data))])
  image_data = image_data.reshape(image_data.shape[0], image_data.shape[1], image_data.shape[2], 1)

  label_data = np.array([obj_data[image_loc][1] for image_loc in range(len(obj_data))])
  label_data = encoder.fit_transform(label_data)

  X_train, X_test, y_train, y_test = train_test_split(image_data, label_data, train_size=0.75, random_state=42)

  X_val = X_train[40:]
  y_val = y_train[40:]
  X_train = X_train[:6460]
  y_train = y_train[:6460]

  return X_train, X_test, X_val, y_train, y_test, y_val

In [17]:
x_train, x_test, x_val, y_train, y_test, y_val = data_preprocessing()

IndexError: tuple index out of range

In [ ]:
print(x_train.shape, x_val.shape)
print(y_train.shape, y_val.shape)

# Print the lengths of the each dataset
print(f"\nX_train: {len(x_train)}")
print(f"y_train: {len(y_train)}")
print(f"X_val: {len(x_val)}")
print(f"y_val: {len(y_val)}")
print(f"x_test: {len(x_test)}")
print(f"y_test: {len(y_test)}")

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras import Sequential,optimizers


def create_model(kernel_size: tuple, input_shape: tuple):
  model = Sequential()
  model.add(Conv2D(IMG_WIDTH,
                  kernel_size=kernel_size,
                  strides=(1,1),
                  padding="same",
                  activation="relu",
                  input_shape=input_shape)           
  )
  model.add(MaxPooling2D((2,2)))
  model.add(Conv2D(IMG_WIDTH,
                  kernel_size=kernel_size,
                  strides=(1,1),
                  padding="same",
                  activation="relu",
                  input_shape=input_shape)           
  )
  model.add(MaxPooling2D((2,2)))
  model.add(Conv2D(IMG_WIDTH/2,
                  kernel_size=kernel_size,
                  strides=(1,1),
                  padding="same",
                  activation="relu",
                  input_shape=input_shape)           
  )

  model.add(Flatten())
  model.add(Dense(64, activation='relu')),
  model.add(Dropout(0.4))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer=optimizers.Adadelta(lr=0.0001), loss="sparse_categorical_crossentropy", metrics=['accuracy'])
  # tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  return model

In [ ]:
my_model = create_model((3, 3), (IMG_WIDTH, IMG_HEIGHT, 1))
results = my_model.fit(x_train, y_train,
                    epochs=600,
                    batch_size=16,
                    validation_data = (x_val,y_val))

In [ ]:
plt.plot(results.history["accuracy"], label="Accuracy")
plt.plot(results.history["val_accuracy"], label="val_accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(results.history["loss"], label="Loss")
plt.plot(results.history["val_loss"], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
# Evaluate the performance
my_model.evaluate(x_test, y_test)

In [ ]:
my_model.save("model.h5")

In [1]:
from tensorflow.keras.models import load_model
model = load_model(r'C:\Users\ozgur\Desktop\Final Project\Bitirme Projesi\model\model_90_32_adam_000005.h5')

In [25]:
image = cv2.imread(r"C:\Users\ozgur\Desktop\Final Project\Bitirme Projesi\Fotoğraflar\5_height_and_congruence_of_the_medial_longitudinal_arch\2\2_1.jpg")
image = np.array(image)
cv2.imshow("foto",image/255)

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

In [4]:

frame = cv2.imread(r"C:\Users\ozgur\Desktop\Final Project\Bitirme Projesi\Fotoğraflar\5_height_and_congruence_of_the_medial_longitudinal_arch\2\2_1.jpg")

if(type(frame) == type(None)):
          pass
else:
    read_images = cv2.resize(frame, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_AREA) 
rgb_image = cv2.cvtColor(read_images,cv2.COLOR_RGB2GRAY)
normalized = rgb_image / 255.0
result = model.predict(np.array([normalized]))

NameError: name 'read_images' is not defined

In [2]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
while True:
    # Kameradan bir kare alın
    ret, frame = cap.read()

    # Görüntüyü boyutlandırın ve normalize edin

    resized = cv2.resize(frame, (64,64))
    rgb_image = cv2.cvtColor(resized,cv2.COLOR_RGB2GRAY)
    normalized = rgb_image / 255.0

    # Görüntüyü modele girdi olarak sağlayın
    result = model.predict(np.array([normalized]))
    print(result)
    # En yüksek olasılığa sahip sınıfı bulun
    predicted_class = np.argmax(result[0])
    
    # Sınıf etiketlerini yükleyin
    class_labels = ["5","4","3","2","1"]
    # print(predicted_class)
    # Sınıf etiketini yazdırın
    label = class_labels[predicted_class]
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    # Görüntüyü gösterin
    cv2.imshow("Kamera", frame)

    # Çıkış için 'q' tuşuna basın
    if cv2.waitKey(1) == ord('q'):
        break

# Kamerayı serbest bırakın ve pencereyi kapatın
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 1s 532ms/step
[[2.4629719e-06 5.2318583e-06 2.6824096e-01 5.4766625e-01 1.8408506e-01
  8.2000655e-09 2.8203160e-09 9.6293676e-12 1.6754395e-10 3.2502776e-09]]
1/1 [==============================] - 0s 48ms/step
[[5.8633144e-05 5.9531434e-05 4.5258048e-01 4.0526247e-01 1.4203817e-01
  4.2484336e-07 1.7257109e-07 3.4184984e-09 2.0327242e-08 1.7373426e-07]]
1/1 [==============================] - 0s 56ms/step
[[9.5105454e-05 8.5582382e-05 3.8824290e-01 4.3492734e-01 1.7664747e-01
  8.3004591e-07 3.6900323e-07 7.9558049e-09 5.2034743e-08 3.6267070e-07]]
1/1 [==============================] - 0s 64ms/step
[[9.9614539e-05 8.7780005e-05 4.0665105e-01 4.2614183e-01 1.6701806e-01
  8.5797296e-07 3.7820985e-07 8.5986667e-09 5.3481070e-08 3.6910211e-07]]
1/1 [==============================] - 0s 32ms/step
[[1.06515006e-04 9.04819535e-05 4.00976896e-01 4.28870708e-01
  1.69953600e-01 9.19021033e-07 4.09854181e-07 9.43281808e-09
  5.94438099e-08 3.96710476e-07